In [25]:
import re
import pandas as pd
from konlpy.tag import Okt

In [26]:
# 검색 키워드들(== 등록된 상품명들)
# TODO 추후 redis로 옮기는 것 고려
registered_goods = [
    '가죽 자켓', '청 자켓', '후드', '맥코트', '신발', '바지', '선글라스', '(메시)니트', '반팔티', '이너', '아우터', 
    '데님 자켓', '데님 팬츠', '데님 트러커', '자켓', '워크자켓', '아디다스 저지', '아디다스 트랙탑', '스카프', '토니웩', '모모타로진'
]

# Okt 객체 생성
okt = Okt()
df = pd.read_csv('../../data/csv/36dc8bba-c0ae-475c-8402-04a1d716b647.csv', header=None, names=["id", "start", "end", "text"])

df.head()

,id,start,end,text
0,5e5d6762-da46-499b-a6ba-b4ab6abb8537,00:00:00,00:00:02,잘 입을 수 있는 그런 자켓인 것 같고
1,2203704f-8918-4bfb-8142-f350c858885e,00:00:02,00:00:03,계절 걱정 많이 하셨잖아요
2,d7cdcc34-527f-47b5-acb7-4aef762e8fd1,00:00:03,00:00:06,가죽 자켓이나 특히 뭐 청자켓 가죽 자켓은
3,8526180f-d687-43c4-af87-994f7566bda6,00:00:06,00:00:08,잠깐 입고 못 입는 거 아니에요?
4,c5fb4e98-48d3-48ca-8ce2-3c7649b5711d,00:00:08,00:00:10,이렇게 얘기를 많이 하시는데


In [ ]:
# TODO 중복으로 등장하는 키워드가 있는 경우 처음 시간대만 체크하는 것 고려
# TODO 해당 코드 이해

# 정규식 패턴
pattern = re.compile("|".join(map(re.escape, registered_goods)))

# 매칭된 키워드 리스트를 새로운 컬럼으로 추가
df["matched_keywords"] = df["text"].str.findall(pattern)

df

,id,start,end,text,matched_keywords
0,5e5d6762-da46-499b-a6ba-b4ab6abb8537,00:00:00,00:00:02,잘 입을 수 있는 그런 자켓인 것 같고,[자켓]
1,2203704f-8918-4bfb-8142-f350c858885e,00:00:02,00:00:03,계절 걱정 많이 하셨잖아요,[]
2,d7cdcc34-527f-47b5-acb7-4aef762e8fd1,00:00:03,00:00:06,가죽 자켓이나 특히 뭐 청자켓 가죽 자켓은,"[가죽 자켓, 자켓, 가죽 자켓]"
3,8526180f-d687-43c4-af87-994f7566bda6,00:00:06,00:00:08,잠깐 입고 못 입는 거 아니에요?,[]
4,c5fb4e98-48d3-48ca-8ce2-3c7649b5711d,00:00:08,00:00:10,이렇게 얘기를 많이 하시는데,[]
5,8cc30a0f-6012-4261-898f-17a8c6a1beae,00:00:10,00:00:12,결국 이게 레이어드거든,[]
6,09a3320d-208a-47e2-8e35-3c52ec1db9bc,00:00:12,00:00:13,그러니까 겨울 되면은,[]
7,f928232c-7e25-4d84-b568-6854106b575f,00:00:13,00:00:15,그냥 더더욱 가죽 자켓 입고 있지만,[가죽 자켓]
8,6b7d6d48-0851-4876-abdf-34856ef290b2,00:00:15,00:00:17,이거 위에 피시텔 같은 거 하나 입는 거예요,[]
9,0c656daa-ddb7-4621-b80b-40d6819b452e,00:00:17,00:00:19,그러니까 그냥 이너로 생각해서 입을...,[이너]


In [38]:
# 매칭된 게 있는 row만 추출
matched_df = df[df["matched_keywords"].str.len() > 0]

print(matched_df)

                                     id      start        end  \
0  5e5d6762-da46-499b-a6ba-b4ab6abb8537   00:00:00   00:00:02   
2  d7cdcc34-527f-47b5-acb7-4aef762e8fd1   00:00:03   00:00:06   
7  f928232c-7e25-4d84-b568-6854106b575f   00:00:13   00:00:15   
9  0c656daa-ddb7-4621-b80b-40d6819b452e   00:00:17   00:00:19   

                       text    matched_keywords  
0     잘 입을 수 있는 그런 자켓인 것 같고                [자켓]  
2   가죽 자켓이나 특히 뭐 청자켓 가죽 자켓은  [가죽 자켓, 자켓, 가죽 자켓]  
7       그냥 더더욱 가죽 자켓 입고 있지만             [가죽 자켓]  
9    그러니까 그냥 이너로 생각해서 입을...                [이너]  
